In [22]:
%matplotlib notebook
import numpy as np
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage import io as skio
from skimage import color, filters
from skimage.morphology import watershed

In [23]:
def bbox1(img):
    a = np.where(img != 0)
    bbox = np.min(a[0]), np.max(a[0]), np.min(a[1]), np.max(a[1])
    return bbox

In [24]:
for i in range(502):
    url = 'batch/France/'+str(i)+'.jpg'
    # url = 'batch/A_curious_herbal/2.jpg'
    img = skio.imread(url)

    gray = color.rgb2gray(img)
    sobel = filters.sobel(gray)

    # from skimage import feature
    # edges1 = feature.canny(gray)
    # edges2 = feature.canny(gray, sigma=3)
    # plt.figure()
    # skio.imshow(edges1)
    # plt.figure()
    # skio.imshow(edges2)
    # plt.imsave("t.jpg",edges2,cmap=plt.cm.gray_r)

    markers = np.zeros_like(gray)
    markers[gray < 0.6] = 2 #黑色部分
    markers[gray > 0.9] = 1 #白色部分

    segmentation = watershed(sobel, markers) 

    img_alpha = np.dstack((img,(segmentation-1).astype(np.uint8)*255)) #加入alpha通道

    ## 寻找面积最大的label
    label_objects, nb_labels = ndi.label(segmentation-1) 
    sizes = np.bincount(label_objects.ravel())
    sizes[0]=0 #去掉0号label（空白）
    mask_sizes = sizes == max(sizes)
    img2 = np.dstack((img,(mask_sizes[label_objects]).astype(np.uint8)*255)) #仅保留最大的部分
    a,b,c,d=bbox1((mask_sizes[label_objects]).astype(np.uint8))
    img3=img2[a:b+1,c:d+1] #切白边
    skio.imsave("france/"+str(i)+".png",img3)